In [3]:
#get dataframe of tweet_id and user_id
import pandas
import csv

df = pandas.read_csv('gamergate_tweets.csv')

In [9]:
import sys
import time
sys.path.append("/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages")

from twython import Twython

#get first 5000 tweet_ids
CONSUMER_KEY = "48r56upQvsnTiSDfY4CNwIpCQ"
CONSUMER_SECRET = "VqZhFW7GqogdKXJnpuS0umVe211bFgoCdaQlRdRMayvrbdDh3k"
OAUTH_TOKEN = "18000528-Scl8a1pLinqP4KWtaa3f8EN0M1vqAGTyYXMtsLIGm"
OAUTH_TOKEN_SECRET = "NcIvLeqVbwwBegabvo7946FVIaDN0CAaJ7aS2aV7pZFzC"

twitter = Twython(
    CONSUMER_KEY, CONSUMER_SECRET,
    OAUTH_TOKEN, OAUTH_TOKEN_SECRET)


start = 0
end = 100
id_list = df["tweet_id"].tolist()
tweet_list = []
while end < len(df):
    
    remaining = twitter.get_application_rate_limit_status()['resources']['statuses']['/statuses/lookup']['remaining']
    if remaining < 2:
        print "start waiting ", start
        time.sleep(920)
        print "done waiting"
        
    twitter = Twython(
    CONSUMER_KEY, CONSUMER_SECRET,
    OAUTH_TOKEN, OAUTH_TOKEN_SECRET)
    
    current = id_list[start:end]
    tweet = twitter.lookup_status(id=current)
    start += 100
    end += 100
    for t in tweet:
        each_tweet = []
        each_tweet.append(t['user']['id'])
        each_tweet.append(t["retweet_count"])
        each_tweet.append(t["favorite_count"])
        if t["entities"]['urls'] != []:
            each_tweet.append(1)
        else: each_tweet.append(0)
        if t["entities"]["user_mentions"] == []:
            each_tweet.append([])
        else: 
            y = 0
            mentionlist = []
            while y < len(t["entities"]["user_mentions"]):
                mentionlist.append((t["entities"]["user_mentions"])[y].items()[2][1])
                y+=1
            each_tweet.append(mentionlist)
        each_tweet.append(t["text"])
        tweet_list.append(each_tweet)
        


start waiting  17900
done waiting
start waiting  35800
done waiting
start waiting  53700
done waiting
start waiting  71600
done waiting
start waiting  89500
done waiting
start waiting  107400
done waiting
start waiting  125300
done waiting
start waiting  143200
done waiting
start waiting  161100
done waiting
start waiting  179000
done waiting
start waiting  196900
done waiting
start waiting  214800
done waiting
start waiting  232700
done waiting
start waiting  250600
done waiting
start waiting  268500
done waiting
start waiting  286400
done waiting
start waiting  304300
done waiting


In [10]:
#put into df    
tweets_df = pandas.DataFrame(tweet_list, columns=["User ID", "Retweets", "Favorites", "Has_url", "Mentions", "Text"])

In [11]:
import re
import sys
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [12]:
#Use Vader sentiment analysis to get polarity
tweetlist = tweets_df["Text"].tolist()
sentences = []
for x in tweetlist:
    x = re.sub(r"http\S+","", x)
    x = re.sub(r"#(\w+)","",x)
    x = re.compile('RT :').sub('', x, count=1)
    x = re.compile('RT').sub('', x, count=1)
    x = re.compile('amp').sub('', x, count=1)
    sentences.append(x)

polarity = []
SIA = SentimentIntensityAnalyzer()
for sentence in sentences:
    ss = SIA.polarity_scores(sentence)
    for k in sorted(ss):
        polarity.append(ss[k])

#Remove URL and hashtags and mentions from text
tweets_df = tweets_df.drop('Text', 1)
tweets_df["Text"] = pandas.Series(sentences)


In [13]:
#add column of lexrating reviews
tweets_df["Polarity"] = pandas.Series(polarity)
    

#polaritylist = tweets_df["Polarity"].tolist()
likelist = tweets_df["Favorites"].tolist()
retweetlist = tweets_df["Retweets"].tolist()

#Create Popularity column, where popularity = num(Retweets) + num(Favorites)
x = 0
popularity = []
while x < len(likelist):
    popularity.append(likelist[x] + retweetlist[x])
    x+=1

tweets_df["Popularity"] = pandas.Series(popularity)

#get nummentions column
numMentions = tweets_df["Mentions"].tolist()
mentionsCount = []
for x in numMentions:
    mentionsCount.append(len(x))
    
tweets_df["numMentions"] = pandas.Series(mentionsCount)

print tweets_df


#write to csv
tweets_df.to_csv("all_tweets.csv", header=True, encoding='utf-8', index=False)
    

           User ID  Retweets  Favorites  Has_url  \
0         20708129       363          0        0   
1         20945241         1          0        0   
2        251045973         0          0        0   
3       1091189214         1          0        0   
4       2847688135         0          0        1   
5       2798609018         0          0        0   
6       2492324996       794          0        1   
7         93363347       238          0        1   
8        937178150        27          0        0   
9         36790442         0          0        1   
10      2590712055        33          0        1   
11        16326705       967          0        0   
12       985889652        80          0        1   
13        18746944       238          0        1   
14       162572003         0          1        0   
15       940132086       228          0        0   
16        49534986        19          0        0   
17       171848975         0          0        0   
18        49

In [14]:
import statsmodels.formula.api as smf

mod = smf.ols(formula='Popularity ~ Polarity + numMentions + Has_url + Retweets + Favorites', data=tweets_df)
res = mod.fit()
print(res.summary())


                            OLS Regression Results                            
Dep. Variable:             Popularity   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 4.931e+31
Date:                Mon, 18 Apr 2016   Prob (F-statistic):               0.00
Time:                        07:50:41   Log-Likelihood:             4.9722e+06
No. Observations:              206330   AIC:                        -9.944e+06
Df Residuals:                  206324   BIC:                        -9.944e+06
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------
Intercept   -3.624e-12   3.29e-14   -110.263      